**General Description**

The following notebook contains the code to create, train, validate, and test a rainfall-runoff model using the LSTM-MDN network architecture. The notebook support running experiments in different large-sample hydrology datasets including: CAMELS-GB, CAMELS-US, CAMELS-DE. The details for each dataset can be read from a .yml file.

***Authors:***
- Eduardo Acuña Espinoza (eduardo.espinoza@kit.edu)
- Manuel Alvarez Chaves (manuel.alvarez-chaves@simtech.uni-stuttgart.de)

In [3]:
import datetime
import pickle
import random
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import xarray as xr
from torch.utils.data import DataLoader
from tqdm import tqdm

from hy2dl.datasetzoo import get_dataset
from hy2dl.modelzoo import get_model
from hy2dl.training.loss import loss_nll
from hy2dl.utils.config import Config
from hy2dl.utils.optimizer import Optimizer
from hy2dl.utils.utils import set_random_seed, upload_to_device

# colorblind friendly palette
color_palette = {"observed": "#377eb8","simulated": "#4daf4a"}

Part 1. Initialize information

In [4]:
# Path to .yml file where the experiment settings are stored. The experimet settings can also be defined manually as a dictionary.
path_experiment_settings = "../examples/mdn.yml"
# path_experiment_settings = "../results/LSTMGMM-250-01_NLL-GAUSSIAN_S42_010/config.yml"
# Read experiment settings
config = Config(path_experiment_settings, dev_mode=True)
config.dump()

FileNotFoundError: [Errno 2] No such file or directory: 'results\\LSTMGMM-250-01_NLL-GAUSSIAN_S05_010\\config.yml'

Part 2. Create datasets and dataloaders used to train/validate the model

In [ ]:
# Get dataset class
Dataset = get_dataset(config)

# Dataset training
config.logger.info(f"Loading training data from {config.dataset} dataset")
total_time = time.time()

training_dataset = Dataset(cfg=config, 
                           time_period="training")

training_dataset.calculate_basin_std()
training_dataset.calculate_global_statistics(save_scaler=True)
training_dataset.standardize_data()

config.logger.info(f"Number of entities with valid samples: {len(training_dataset.df_ts)}")
config.logger.info(f"Time required to process {len(training_dataset.df_ts)} entities: {datetime.timedelta(seconds=int(time.time()-total_time))}")
config.logger.info(f"Number of valid training samples: {len(training_dataset)}\n")

# Dataloader training
train_loader = DataLoader(dataset=training_dataset,
                          batch_size=config.batch_size_training,
                          shuffle=True,
                          drop_last=True,
                          collate_fn=training_dataset.collate_fn,
                          num_workers=config.num_workers)

# Print details of a loader´s sample to check that the format is correct
config.logger.info("Details training dataloader".center(60, "-"))
config.logger.info(f"Batch structure (number of batches: {len(train_loader)})")
config.logger.info(f"{'Key':^30}|{'Shape':^30}")

# Loop through the sample dictionary and print the shape of each element
for key, value in next(iter(train_loader)).items():
    if key.startswith(("x_d", "x_conceptual")):
        config.logger.info(f"{key}")
        for i, v in value.items():
            config.logger.info(f"{i:^30}|{str(v.shape):^30}")
    else:
        config.logger.info(f"{key:<30}|{str(value.shape):^30}")
        
config.logger.info("")  # prints a blank line

In [ ]:
config.logger.info(f"Loading validation data from {config.dataset} dataset")

# Validate on random basins
if config.validate_n_random_basins > 0:
    entities_ids = np.loadtxt(config.path_entities, dtype="str").tolist()
    random_basins = np.random.choice(entities_ids, size=config.validate_n_random_basins, replace=False).tolist()

total_time = time.time()
validation_dataset = Dataset(cfg=config,
                      time_period="validation",
                      entities_ids=random_basins if config.validate_n_random_basins > 0 else None)
    
validation_dataset.scaler = training_dataset.scaler
validation_dataset.standardize_data()

config.logger.info(f"Time required to process {len(validation_dataset.df_ts)} entities: {datetime.timedelta(seconds=int(time.time()-total_time))}\n")
config.logger.info(f"Number of validation samples: {len(validation_dataset)}\n")

validation_loader = DataLoader(dataset=validation_dataset,
                                  batch_size=config.batch_size_evaluation,
                                  shuffle=False,
                                  drop_last=False,
                                  collate_fn=validation_dataset.collate_fn,
                                  num_workers=config.num_workers)

In [ ]:
# Initialize model
set_random_seed(cfg=config)
model = get_model(config).to(config.device)

Part 3. Train model

In [ ]:
# Initialize model
set_random_seed(cfg=config)
model = get_model(config).to(config.device)

# Initialize optimizer
optimizer = Optimizer(cfg=config, model=model) 

# Training report structure
config.logger.info("Training model".center(60, "-"))
config.logger.info(f"{'':^16}|{'Trainining':^21}|{'Validation':^21}|")
config.logger.info(f"{'Epoch':^5}|{'LR':^10}|{'Loss':^10}|{'Time':^10}|{'Metric':^10}|{'Time':^10}|")

total_time = time.time()
# Loop through epochs
for epoch in range(1, config.epochs + 1):
    train_time = time.time()
    loss_evol = []
    # Training -------------------------------------------------------------------------------------------------------
    model.train()
    # Loop through the different batches in the training dataset
    iterator = tqdm(train_loader, 
                    desc=f"Epoch {epoch}/{config.epochs}. Training", 
                    unit="batches", 
                    ascii=True, 
                    leave=False)
    
    for idx, sample in enumerate(iterator):
        # reach maximum iterations per epoch
        if config.max_updates_per_epoch is not None and idx >= config.max_updates_per_epoch:
            break

        sample = upload_to_device(sample, config.device)  # upload tensors to device
        optimizer.optimizer.zero_grad()  # sets gradients to zero
        
        # Forward pass of the model
        pred = model(sample)
        loss = loss_nll(
            params=pred["params"],
            weights=pred["weights"],
            dist=model.distribution,
            y_obs=sample["y_obs"]
        )
        loss = loss.sum()
        
        # Backpropagation (calculate gradients)
        loss.backward()
        
        # Update model parameters (e.g, weights and biases)
        optimizer.clip_grad_and_step(epoch, idx)

        # Keep track of the loss per batch
        loss_evol.append(loss.item())
        iterator.set_postfix({"loss": f"{np.mean(loss_evol):.3f}"})

        # remove elements from cuda to free memory
        del sample, pred
        torch.cuda.empty_cache()

    # training report
    report = f'{epoch:^5}|{optimizer.optimizer.param_groups[0]["lr"]:^10.5f}|{np.mean(loss_evol):^10.3f}|{str(datetime.timedelta(seconds=int(time.time()-train_time))):^10}|'

    # Validation -----------------------------------------------------------------------------------------------------
    if epoch % config.validate_every == 0:
        val_time = time.time()
        model.eval()

        iterator = tqdm(validation_loader, 
                desc=f"Epoch {epoch}/{config.epochs}. Validation", 
                unit="batches", 
                ascii=True, 
                leave=False)
        
        with torch.no_grad():
            loss_evol = []
            for idx, sample in enumerate(iterator):
                sample = upload_to_device(sample, config.device)
                pred = model(sample)
                loss = loss_nll(
                    params=pred["params"],
                    weights=pred["weights"],
                    dist=model.distribution,
                    y_obs=sample["y_obs"]
                )
                loss = loss.sum()
                loss_evol.append(loss.item())
                iterator.set_postfix({"loss": f"{np.nanmean(loss_evol):.3f}"})

            # average loss validation
            report += f"{np.nanmean(loss_evol):^10.3f}|{str(datetime.timedelta(seconds=int(time.time()-val_time))):^10}|"

    # No validation
    else:
        report += f"{'':^10}|{'':^10}|"
    

    # Print report and save model
    config.logger.info(report)
    torch.save(model.state_dict(), config.path_save_folder / "model" / f"model_epoch_{epoch}")
    # modify learning rate
    optimizer.update_optimizer_lr(epoch=epoch)

# print total training time
config.logger.info(f'Total training time: {datetime.timedelta(seconds=int(time.time()-total_time))}\n')

Part 4. Test model

In [ ]:
# In case I already trained an LSTM I can re-construct the model. I just need to define the epoch for which I want to
# re-construct the model
# model = get_model(config).to(config.device)
# model.load_state_dict(torch.load(config.path_save_folder / "model" / "model_epoch_10", map_location=config.device))

In [ ]:
# Read previously generated scaler
with open(config.path_save_folder / "scaler.pickle", "rb") as file:
    scaler = pickle.load(file)

In [ ]:
other_location = config.path_save_folder.parent / "LSTMGMM-250-01_NLL-GAUSSIAN_S42_531"

model = get_model(config).to(config.device)
model.load_state_dict(torch.load(other_location / "model" / "model_epoch_10", map_location=config.device))

with open(other_location / "scaler.pickle", "rb") as file:
    scaler = pickle.load(file)

# Get dataset class
Dataset = get_dataset(config)

In [ ]:
# In evaluation (validation and testing) we will create an individual dataset per basin
config.logger.info(f"Loading testing data from {config.dataset} dataset")

entities_ids = np.loadtxt(config.path_entities_testing, dtype="str").tolist()
iterator = tqdm([entities_ids] if isinstance(entities_ids, str) else entities_ids, 
                desc="Processing entities", 
                unit="entity", 
                ascii=True)

total_time = time.time()
testing_dataset = {}
for entity in iterator:
    dataset = Dataset(cfg= config, 
                      time_period= "testing",
                      check_NaN=False,
                      entities_ids=entity)

    dataset.scaler = scaler
    dataset.standardize_data(standardize_output=False)
    testing_dataset[entity] = dataset

config.logger.info(f"Time required to process {len(iterator)} entities: {datetime.timedelta(seconds=int(time.time()-total_time))}\n")

In [ ]:
config.logger.info("Testing model".center(60, "-"))
total_time = time.time()

model.eval()
out = {}
with torch.no_grad():
    # Go through each basin
    iterator = tqdm(testing_dataset, desc=f"Testing", unit="basins", ascii=True)
    for basin in iterator:
        loader = DataLoader(
            dataset=testing_dataset[basin],
            batch_size=config.batch_size_evaluation,
            shuffle=False,
            drop_last=False,
            collate_fn=testing_dataset[basin].collate_fn,
            num_workers=config.num_workers
        )

        dates, y_obs, y_hat, params, weights = [], [], [], {}, []
        for sample in loader:
            sample = upload_to_device(sample, config.device)  # upload tensors to device

            dates.append(sample["date"])
            y_obs.append(sample["y_obs"].detach().cpu().numpy())

            # Generate predictions
            batch_params, batch_weights = model(sample).values()
            for k, v in batch_params.items():
                params[k] = params.get(k, []) + [v.detach().cpu().numpy()]

            weights.append(batch_weights.detach().cpu().numpy())

            pred = model.sample(sample, 1)
            pred[:, :, 0, :] = model.mean(sample)

            # backtransformed information
            pred = pred * testing_dataset[basin].scaler["y_std"].to(config.device) + (testing_dataset[basin].scaler["y_mean"].to(config.device))
            y_hat.append(pred.detach().cpu().numpy())

            # remove from cuda
            del sample, pred
            torch.cuda.empty_cache()

        out[basin] = {
            "date": np.concatenate(dates),
            "y_obs": np.concatenate(y_obs),
            "y_hat": np.concatenate(y_hat),
            "params": {k: np.concatenate(v) for k, v in params.items()},
            "weights": np.concatenate(weights)
        }
        del dates, y_obs, y_hat, params, weights


config.logger.info(f'Total testing time: {datetime.timedelta(seconds=int(time.time()-total_time))}')

In [ ]:
loader = DataLoader(
            dataset=testing_dataset["13011900"],
            batch_size=config.batch_size_evaluation,
            shuffle=False,
            drop_last=False,
            collate_fn=testing_dataset["13011900"].collate_fn,
            num_workers=config.num_workers
        )

In [ ]:
sample = next(iter(loader))

In [ ]:
y_mean = testing_dataset[basin].scaler["y_mean"].to(config.device)
y_std = testing_dataset[basin].scaler["y_std"].to(config.device)

dates = sample["date"][:, -1]
y_obs = sample["y_obs"][:, -1, 0]
mean = (model.mean(sample)[:, -1, 0] * y_std + y_mean)

samples = (model.sample(sample, num_samples=1000)[:, 0, :, 0] * y_std + y_mean).detach()

y_obs_std = (y_obs - y_mean) / y_std
logpdf = model._calc_logpdf(sample, y_obs_std)[:, -1, 0]

cdf = model._calc_cdf(sample, y_obs_std)[:, -1, 0]

In [ ]:
quantile = model.quantile(sample, [0.025, 0.5, 0.975])

In [ ]:
q_025 = (quantile[:, 0, 0, 0] * y_std + y_mean).numpy()
q_50 = (quantile[:, 0, 1, 0] * y_std + y_mean).numpy()
q_975 = (quantile[:, 0, 2, 0] * y_std + y_mean).numpy()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(dates, y_obs, label="Observed", color="C0")
ax.plot(dates, q_50, label="Quantile 0.50", color="C2")
ax.fill_between(dates, q_025, q_975, color="C2", alpha=0.2, label="Quantile 0.025 - 0.975")
ax.set_ylim(-0.5, 17.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(dates, mean - q_50, label="Mean - Quantile 0.50", color="C3")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(dates, y_obs - y_obs, label="Observed", color="C0")
ax.plot(dates, mean - y_obs, label="Predicted", color="C1")
ax.fill_between(dates, samples.quantile(0.025, axis=1) - y_obs, samples.quantile(0.975, axis=1) - y_obs, color="C1", alpha=0.2)
ax.set_ylim(-4, 2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(dates, logpdf.detach().numpy(), label="Log-probability", color="C0")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(dates, cdf.detach().numpy(), label="CDF", color="C0")
plt.show()

In [ ]:
def to_xarray(out_dict):
    # Get all basin IDs
    basin_ids = list(out_dict.keys())
    
    # Get dates from first basin (assuming all basins have same dates)
    dates = out_dict[basin_ids[0]]['date']
    num_days = dates.shape[0]
    
    # Get dimensions from the first basin
    first_basin = out_dict[basin_ids[0]]
    num_basins = len(basin_ids)
    predict_last_n = first_basin['y_obs'].shape[1]
    num_targets = first_basin['y_obs'].shape[2]
    num_samples = first_basin['y_hat'].shape[2]
    
    # Get parameter names dynamically from params subdictionary
    param_names = list(first_basin['params'].keys())
    num_components = first_basin['params'][param_names[0]].shape[2]
    
    # Initialize arrays for fixed variables
    y_obs_array = np.zeros((num_basins, num_days, predict_last_n, num_targets))
    y_hat_array = np.zeros((num_basins, num_days, predict_last_n, num_samples, num_targets))
    weights_array = np.zeros((num_basins, num_days, predict_last_n, num_components, num_targets))
    
    # Initialize arrays for parameter variables dynamically
    param_arrays = {}
    for param_name in param_names:
        param_arrays[param_name] = np.zeros((num_basins, num_days, predict_last_n, num_components, num_targets))
    
    # Fill arrays
    for idx, basin_id in enumerate(basin_ids):
        basin_data = out_dict[basin_id]

        y_obs_array[idx] = basin_data['y_obs']
        y_hat_array[idx] = basin_data['y_hat']
        weights_array[idx] = basin_data['weights']
        
        # Fill parameter arrays dynamically
        for param_name in param_names:
            param_arrays[param_name][idx] = basin_data['params'][param_name]

    # Create coordinate arrays
    coords = {
        'basin_id': basin_ids,
        'date': dates[:, -1],
        'predict_last_n': np.arange(predict_last_n),
        'num_targets': np.arange(num_targets),
        'num_samples': np.arange(num_samples),
        'num_components': np.arange(num_components)
    }
    
    # Create data variables
    data_vars = {
        'y_obs': (['basin_id', 'date', 'predict_last_n', 'num_targets'], y_obs_array),
        'y_hat': (['basin_id', 'date', 'predict_last_n', 'num_samples', 'num_targets'], y_hat_array),
        'weights': (['basin_id', 'date', 'predict_last_n', 'num_components', 'num_targets'], weights_array)
    }
    
    # Add parameter variables dynamically
    for param_name in param_names:
        data_vars[param_name] = (['basin_id', 'date', 'predict_last_n', 'num_components', 'num_targets'], param_arrays[param_name])
    
    # Create the dataset
    ds = xr.Dataset(data_vars, coords=coords)
    
    # Add attributes for better documentation
    ds.attrs['description'] = 'Basin prediction data with observations, predictions, and model parameters'
    ds['y_obs'].attrs['description'] = 'Observed values'
    ds['y_hat'].attrs['description'] = 'Predicted values (ensemble samples)'
    ds['weights'].attrs['description'] = 'Weight values'
    
    # Add parameter descriptions dynamically
    for param_name in param_names:
        ds[param_name].attrs['description'] = f'{param_name.capitalize()} parameters'
    
    return ds

ds = to_xarray(out)
path_experiment = config.path_save_folder / f"rest_results.nc"
compression = {var: {"zlib":True, "complevel":5, "dtype":"f4"} for var in ds.data_vars}
ds.to_netcdf(path_experiment, encoding=compression)

Part 5. Initial analysis

In [ ]:
# Select data
basin_to_analyze = "13011900"
data = ds.sel(basin_id=basin_to_analyze).sel(date=slice("1991-02-20", "1991-11-15"))

In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=(10, 8))

for idx, ax in enumerate(axes):
    ax.plot(data["date"], data["y_obs"][:, 0, idx], label="Observed", color=color_palette["observed"])
    ax.plot(data["date"], data["y_hat"].median(dim="num_samples")[:, 0, idx], label="Simulated", color=color_palette["simulated"])
    ax.fill_between(data["date"], data["y_hat"].quantile(0.05, dim="num_samples")[:, 0, idx], data["y_hat"].quantile(0.95, dim="num_samples")[:, 0, idx], color=color_palette["simulated"], alpha=0.35)
    ax.grid(ls="--", alpha=0.5)

# Format plot
axes[0].legend()
axes[0].set_ylabel("Streamflow [mm/day]")
axes[1].set_ylabel("Vapor pressure [Pa]")
axes[1].set_xlabel("Date")
plt.show()